In [ ]:
import bw2data as bd
import bw2io as bi
import bw2regional as bwr
import bw2calc as bc
import bw2analyzer as ba

Here if our matrix equation:

$ h_{r} = [ MN_{dx}DXN_{g}GR ]^{T} \cdot [ BA^{-1}f ]$

In [ ]:
bd.projects.set_current("LC IMPACT case study")

In [ ]:
meal = bd.Database('meal')
meal.process()

In [ ]:
my_method = ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'All')

Start the calculations

In [ ]:
def combine_xts(demand, method, xtables):    
    first, others = xtables[0], xtables[1:]
    
    lca = bwr.ExtensionTablesLCA(
        demand=demand,
        method=method,
        xtable=first
    )
    lca.lci()
    lca.lcia()
    
    for xtable in others:
        other_lca = bwr.ExtensionTablesLCA(
            demand=demand,
            method=method,
            xtable=xtable
        )
        other_lca.lci()
        other_lca.lcia()
        
        lca.characterized_inventory += other_lca.characterized_inventory
    
    return lca

In [ ]:
xtables = [
    'alm-xt',     
    'bcn-xt',
    'world-topo-watersheds-eq-sw-all-weighted-pop-density',
]

In [ ]:
lca = combine_xts({('meal', 'meal'): 1}, my_method, xtables)   

In [ ]:
lca.geodataframe_inv_spatial_scale(cutoff=None).explore(column='score_rel')

Contribution (can be regionalized)

In [ ]:
ba.ContributionAnalysis(max_level=4).annotated_top_processes(lca, names=True)

Compare with site-generic LCA

In [ ]:
sg = bc.LCA(
    demand={('meal', 'meal'): 1},
    method=my_method,
)
sg.lci()
sg.lcia()
sg.score

Recursive calculation (only site-generic)

In [ ]:
ba.utils.print_recursive_calculation(('meal', 'meal'), my_method, max_level=4)